In [4]:
#구글 애플리케이션을 등록해서 구글 캘린더 api를 사용하고,
#네이버 애플리케이션에 등록해서 지역 검색 api를 사용


import requests

#네이버 애플리케이션의 client_id와 client_secret키 설정
headers = {
    "X-Naver-Client-Id" : "8ROaw9rUz_iiYI5QyPR_",
    "X-Naver-Client-Secret" : "YumdWGYhgG"
}

#지역 검색 요청 마라미터 설정
query = "건대 맛집"
params = {
    "sort" : "comment",
    "query": query,
    "display": 3
}

#지역 검색 url과 요청 파라미터
naver_local_url = "https://openapi.naver.com/v1/search/local.json"

#지역 검색 요청
res = requests.get(naver_local_url, headers=headers, params=params)
if res.status_code == 200:
    #지역 검색 결과 확인
    places = res.json().get('items')
    print(places)

[{'title': '북카페세모', 'link': 'http://cafe.naver.com/semoclass/', 'category': '음식점>카페,디저트', 'description': '', 'telephone': '', 'address': '서울특별시 광진구 화양동 8-43', 'roadAddress': '서울특별시 광진구 아차산로31길 20 2층, 3층', 'mapx': '317993', 'mapy': '549269'}, {'title': '호야초밥참치 본점', 'link': '', 'category': '일식>초밥,롤', 'description': '', 'telephone': '', 'address': '서울특별시 광진구 화양동 10-1 1층', 'roadAddress': '서울특별시 광진구 능동로13길 39 1층', 'mapx': '317995', 'mapy': '549494'}, {'title': '스시오 <b>건대</b>점', 'link': 'https://sushiogundae.modoo.at', 'category': '음식점>일식,초밥뷔페', 'description': '', 'telephone': '', 'address': '서울특별시 광진구 화양동 3-22', 'roadAddress': '서울특별시 광진구 능동로 141', 'mapx': '318195', 'mapy': '549437'}]


In [20]:
#구글캘린더 인증->사용자 인증 받기

#필요한 라이브러리
from google_auth_oauthlib.flow import InstalledAppFlow
#구글 캘린더 API 서비스 객체 생성
from googleapiclient.discovery import build
import datetime

#구글 클라우드 콘솔에서 다운받은 OAuth 2.0 클라이언트 파일 경로
creds_filename = 'D:/22KJH/project/Python/8smart_scheduler/google_token.json'
#사용 권한 지정
#https://www.googleapis.com/auth/calendar.readonly 하면 읽기 권한만 허용
SCOPES = ['https://www.googleapis.com/auth/calendar'] #읽기, 쓰기 권한 허용

#파일에 담긴 인증 정보로 구글 서버에 인증하기
#새 창이 열리면서 구글 로그인 및 정보 제공 동의 후 최종 인증이 완료
flow = InstalledAppFlow.from_client_secrets_file(creds_filename, SCOPES)
creds = flow.run_local_server(port=0)

#구글에서 제공하는 클라이언트를 사용해 서비스 객체 생성
service = build('calendar', 'v3', credentials=creds)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=675755398021-rjos50chpg4djpmkl11ln0pm1q6lcgjo.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A52107%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=XoKX8mAe7EOqLyvkFcDAIjH5U1smUX&access_type=offline


In [14]:
#일정을 생성, 조회, 수정, 삭제(CRUD) 하기

##일정 생성(Create)
#일정을 생성할 날짜
today = datetime.datetime.today().strftime("%Y-%m-%d")

event = {
    'summary': '오늘 배워 오늘 쓰는 OpenAPI 수업', # 일정 제목
    'location': '서울특별시 성북구 정릉동 정릉로 77', # 일정 장소
    'description': 'OpenAPI 수업에 대한 설명입니다.', #일정 설명
    'start': { #시작 날짜
        'dateTime': today + 'T09:00:00',
        'timeZone': 'Asia/Seoul',
    },
    'end': { # 종료 날짜
        'dateTime': today + 'T10:00:00',
        'timeZone': 'Asia/Seoul',
    },
    'attendees': [ #참석자
        {'email': 'lpage#example@com'},
        {'email': 'sbrin#example@com'},
    ],
    'reminders': { #알림 설정
        'useDefault': False,
        'overrides': [
            {'method': 'email','minutes':24*60}, #24*60 = 하루 전 이메일 알림
            {'method': 'popup','minutes':10}, #10분 전 팝업 알림
        ],
    },
}

#calendarId: 캘린더 ID/ primary가 기본값
event = service.events().insert(calendarId='primary', body=event).execute()
print('Event created: %s' % (event.get('htmlLink')))

Event created: https://www.google.com/calendar/event?eid=djFpMTFxZHIwczBsbmVicjNzcXU1Ymd2NTQganVuZ2h5dW5lZTI2QG0


In [17]:
##일정 조회(Road)
#조회에 사용될 요청 변수 지정
calendar_id = 'primary'

time_min = today + 'T00:00:00+09:00' #일정을 조회할 최소 날짜
time_max = today + 'T23:59:59+09:00:00' #일정을 조회할 최대 날짜
max_results = 5 #조회할 최대 일정 개수
is_single_events = True #반복 일정의 여부
orderby = 'startTime' #일정 정렬 방식

events_result = service.events().list(calendarId = calendar_id,
                                    timeMin = time_min,
                                    timeMax = time_max,
                                    maxResults = max_results,
                                    singleEvents = is_single_events,
                                    orderBy = orderby
                                    ).execute()

items = events_result.get('items')
print("===== [일정 목록 출력] =====")
print(items)

===== [일정 목록 출력] =====
[{'kind': 'calendar#event', 'etag': '"3285504051426000"', 'id': 'dbmo2daats6905k18af2pjssec', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=ZGJtbzJkYWF0czY5MDVrMThhZjJwanNzZWMganVuZ2h5dW5lZTI2QG0', 'created': '2022-01-21T08:00:25.000Z', 'updated': '2022-01-21T08:00:25.747Z', 'summary': '오늘 배워 오늘 쓰는 OpenAPI 수업', 'description': 'OpenAPI 수업에 대한 설명입니다. 정말 재밌습니다.', 'location': '서울특별시 성북구 정릉동 정릉로 77', 'creator': {'email': 'junghyunee26@gmail.com', 'self': True}, 'organizer': {'email': 'junghyunee26@gmail.com', 'self': True}, 'start': {'dateTime': '2022-01-21T00:00:00Z', 'timeZone': 'Asia/Seoul'}, 'end': {'dateTime': '2022-01-21T01:00:00Z', 'timeZone': 'Asia/Seoul'}, 'iCalUID': 'dbmo2daats6905k18af2pjssec@google.com', 'sequence': 0, 'attendees': [{'email': 'sbrin@example.com', 'responseStatus': 'needsAction'}, {'email': 'lpage@example.com', 'responseStatus': 'needsAction'}], 'reminders': {'useDefault': False, 'overrides': [{'method': 'ema

In [18]:
##일정 수정(Update)
event = events_result.get('items')[0] #events_result를 리스트로 만들었으니까 [0]로 찾음
event_id = event.get('id') #아이디값을 가져와서(어떤 일정인지를 지정)

#원하는 일정(list[0])의 속성을 수정
event['summary'] = "(수정된)" + event['summary']

#일정 수정 요청하기
updated_event = service.events().update(calendarId='primary', eventId = event_id, body = event).execute()

In [19]:
##일정 삭제(Delete)
#eventId: 일정을 조회한 후 얻은 id 값
eventId = updated_event.get('id')
service.events().delete(calendarId ='primary', eventId = event_id).execute()

''

In [21]:
#구현하기

#필요한 라이브러리
from google_auth_oauthlib.flow import InstalledAppFlow
#구글 캘린더 API 서비스 객체 생성
from googleapiclient.discovery import build
import datetime

#구글 클라우드 콘솔에서 다운받은 OAuth 2.0 클라이언트 파일 경로
creds_filename = 'D:/22KJH/project/Python/8smart_scheduler/google_token.json'
#사용 권한 지정
#https://www.googleapis.com/auth/calendar.readonly 하면 읽기 권한만 허용
SCOPES = ['https://www.googleapis.com/auth/calendar'] #읽기, 쓰기 권한 허용

#파일에 담긴 인증 정보로 구글 서버에 인증하기
#새 창이 열리면서 구글 로그인 및 정보 제공 동의 후 최종 인증이 완료
flow = InstalledAppFlow.from_client_secrets_file(creds_filename, SCOPES)
creds = flow.run_local_server(port=0)

#구글에서 제공하는 클라이언트를 사용해 서비스 객체 생성
service = build('calendar', 'v3', credentials=creds)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=675755398021-rjos50chpg4djpmkl11ln0pm1q6lcgjo.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A52117%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=GVC3zdhxHCmyyg8AgSozoYLeP4od5e&access_type=offline


In [27]:
#조회에 사용될 요청 변수 지정
calendar_id = 'primary'
today = datetime.datetime.today().strftime("%Y-%m-%d")

time_min = today + 'T00:00:00+09:00' #일정을 조회할 최소 날짜
time_max = today + 'T23:59:59+09:00:00' #일정을 조회할 최대 날짜
max_results = 5 #조회할 최대 일정 개수
is_single_events = True #반복 일정의 여부
orderby = 'startTime' #일정 정렬 방식

#오늘의 일정 가져오기
events_result = service.events().list(calendarId = calendar_id,
                                    timeMin = time_min,
                                    timeMax = time_max,
                                    maxResults = max_results,
                                    singleEvents = is_single_events,
                                    orderBy = orderby
                                    ).execute()

items = events_result.get('items')
print("===== [일정 목록 출력] =====")
print(items)
item = items[2] #테스트를 위해 오늘 일정 중 하나만 가져옴

#일정 제목
gsummary = item.get('summary')

#일정 제목에서 [식사-국민대]에서 카테고리-장소 를 -를 기준으로 추출
gcategory, glocation = gsummary[gsummary.index('[')+1:gsummary.index(']')].split('-')

#구글 캘린더 일정이 연결되어 있는 링크
gevent_url = item.get('htmlLink')

print("\n\n ===== [일정 상세 정보 출력] =====")
print("category: ", gcategory)
print("location: ", glocation)
print("event_url: ", gevent_url)

===== [일정 목록 출력] =====
[{'kind': 'calendar#event', 'etag': '"3285504702105000"', 'id': 'k30tcohvu1qupboipn79cj6vqs', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=azMwdGNvaHZ1MXF1cGJvaXBuNzljajZ2cXMganVuZ2h5dW5lZTI2QG0', 'created': '2022-01-21T08:05:50.000Z', 'updated': '2022-01-21T08:05:51.091Z', 'summary': '오늘 배워 오늘 쓰는 OpenAPI 수업', 'description': 'OpenAPI 수업에 대한 설명입니다. 정말 재밌습니다.', 'location': '서울특별시 성북구 정릉동 정릉로 77', 'creator': {'email': 'junghyunee26@gmail.com', 'self': True}, 'organizer': {'email': 'junghyunee26@gmail.com', 'self': True}, 'start': {'dateTime': '2022-01-21T09:00:00+09:00', 'timeZone': 'Asia/Seoul'}, 'end': {'dateTime': '2022-01-21T10:00:00+09:00', 'timeZone': 'Asia/Seoul'}, 'iCalUID': 'k30tcohvu1qupboipn79cj6vqs@google.com', 'sequence': 0, 'attendees': [{'email': 'sbrin@example.com', 'responseStatus': 'needsAction'}, {'email': 'lpage@example.com', 'responseStatus': 'needsAction'}], 'reminders': {'useDefault': False, 'overrides': [{'met

In [32]:
##네이버 지역 검색으로 맛집 검색
import requests

#네이버 애플리케이션의 client_id와 client_secret키 설정
headers = {
    "X-Naver-Client-Id" : "8ROaw9rUz_iiYI5QyPR_",
    "X-Naver-Client-Secret" : "YumdWGYhgG"
}

#지역 검색 요청 마라미터 설정
query = glocation + "맛집"
params = {
    "sort" : "comment",
    "query": query,
    "display": 3
}

#지역 검색 url과 요청 파라미터
naver_local_url = "https://openapi.naver.com/v1/search/local.json"

#지역 검색 요청
res = requests.get(naver_local_url, headers=headers, params=params)

#지역 검색 결과 확인
places = res.json().get('items')
print(places)

[{'title': '북카페세모', 'link': 'http://cafe.naver.com/semoclass/', 'category': '음식점>카페,디저트', 'description': '', 'telephone': '', 'address': '서울특별시 광진구 화양동 8-43', 'roadAddress': '서울특별시 광진구 아차산로31길 20 2층, 3층', 'mapx': '317993', 'mapy': '549269'}, {'title': '호야초밥참치 본점', 'link': '', 'category': '일식>초밥,롤', 'description': '', 'telephone': '', 'address': '서울특별시 광진구 화양동 10-1 1층', 'roadAddress': '서울특별시 광진구 능동로13길 39 1층', 'mapx': '317995', 'mapy': '549494'}, {'title': '스시오 <b>건대</b>점', 'link': 'https://sushiogundae.modoo.at', 'category': '음식점>일식,초밥뷔페', 'description': '', 'telephone': '', 'address': '서울특별시 광진구 화양동 3-22', 'roadAddress': '서울특별시 광진구 능동로 141', 'mapx': '318195', 'mapy': '549437'}]


In [35]:
##카카오톡 메세지 보내기

import json
import kakao_utils

KAKAO_TOKEN_FILENAME = "D:/22KJH/project/Python/7summarize_news/kakao_token.json"
KAKAO_APP_KEY = "5667bb7ce28bd6798e5f51bd12b0c0fc"
kakao_utils.update_tokens(KAKAO_APP_KEY, KAKAO_TOKEN_FILENAME)

{'access_token': 'o49xT-xUNQE5Z2ngUpBzfosV0mnbKIvuEVfV_wo9dRsAAAF-e-Z6Uw', 'token_type': 'bearer', 'expires_in': 21599}


{'access_token': 'o49xT-xUNQE5Z2ngUpBzfosV0mnbKIvuEVfV_wo9dRsAAAF-e-Z6Uw',
 'token_type': 'bearer',
 'refresh_token': '3NMYGq_hOmv-n4vzc4x6CaGlikz9gjGVuvBk8QopyV4AAAF-dgXVpg',
 'expires_in': 21599,
 'scope': 'account_email profile_image talk_message profile_nickname',
 'refresh_token_expires_in': 5183999}

In [41]:
#일정 주소 네이버 연결할 링크
gaddr_url = "https://search.naver.com/search.naver?query=" + glocation + "맛집"
#contents 변수를 초기화
contents = []

#카카오톡 리스트 템플릿 작성
template = {
    "object_type": "list",
    "header_title" : gsummary + "- 맛집 추천",
    "header_link": {
        "web_url": gevent_url,
        "mobile_web_url": gevent_url
    },
    "contents": contents,
    "buttons": [
        {
            "title": "일정 자세히 보기",
            "link": {
                "web_url": gevent_url,
                "mobile_web_url": gevent_url
            }
        },
        {
            "title": "일정 장소 보기",
            "link": {
                "web_url": gaddr_url,
                "mobile_web_url": gaddr_url
            }
        }
    ],
}

#카카오톡 리스트 템플릿의 contents를 구성
for place in places:
    ntitle = place.get('title') #장소 이름
    ncategory = place.get('category') #장소 카테고리
    ntelephone = place.get('telephone') #장소 전화번호
    nlocation = place.get('address') #장소 지번 주소
    
    #각 장소를 클릭할 때 네이버 검색으로 연결해주기
    query = nlocation + ' ' + ntitle
    
    #장소 카테고리가 카페면 카페 이미지/ 그 외에는 음식 이미지
    if '카페' in ncategory:
        image_url = "https://freesvg.org/img/pitr_Coffee_cup_icon.png"
    else:
        image_url = "https://freesvg.org/img/bentolunch.png?w=150&h=150&fit=fill"
        
    #전화번호가 있다면 제목과 함께 넣어준다.
    if ntelephone:
        ntitle = ntitle + "\ntel)" + ntelephone
        
    #카카오톡 리스트 템플릿 형식에 맞춰줌
    content = {
        "title": "[" + ncategory + "]" + ntitle,
        "description": ' '.join(nlocation.split('-')[1:]),
        "image_url" : "https://freesvg.org/img/pitr_Coffee_cup_icon.png",
        "image_width" : 50, "image_height" : 50,
        "link" : {
            "web_url": "https://search.naver.com/search.naver?query=" + query,
            "mobie_web_url" : "https://search.naver.com/search.naver?query=" + query
        }
    }
    contents.append(content)
    
#카톡 전송
res = kakao_utils.send_message(KAKAO_TOKEN_FILENAME, template)
if res.json().get('result_code') == 0:
    print('일정 맞춤 맛집을 성공적으로 보냈습니다.')
else:
    print('일정 맞춤 맛집을 성공적으로 보내지 못했습니다. 오류메세지: ',  res.json())

일정 맞춤 맛집을 성공적으로 보냈습니다.
